# CNN for Image Classification Task

### M32895 Course Work "Big Data" 2025

## Notebook objectives
* Build, train and test a convolutional neural network (CNN) for Classification task using an image dataset

## Inputs

## Outputs

## Notes and comments

---

### Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

### Settings

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # tf will show error messages only (reduce verbosity)
sns.set_style('white')

---

### <u>Load data</u>

In [ ]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
print(X_train.shape, X_test.shape)
# The dataset is already preprocessed and split into train and test

In [ ]:
n_labels = len(np.unique(y_train))
n_labels

#### Workflow for data acquisition

* Load the data first. We are using the fashion_mnist dataset from tensorflow. It has 28x28 graiscale images of clothes and we are interested in predicting which piece of close it is.
* The data are split into train and test sets (It is available from TensorFlow in this format).
* This is a demo dataset, where all images are provided in a single and standardized format and arranged in a NumPy array.
* This is useful for learning purposes. However, actual image datasets rarely have the characteristic of having all images of the same size.

In [ ]:
pointer = 15 # number of image in the dataset (remember, numbering starts from 0!)

print(f"array pointer = {pointer}")
print(f"x_train[{pointer}] shape: {X_train[pointer].shape}")
print(f"label: {y_train[pointer]}")

plt.imshow(X_train[pointer],cmap='Accent')
plt.show()

#### Fashion MNIST dataset labels

![Fashion MNIST dataset labels](../assets/img/fashion_mnist_labels.png)

---

### <u>Data preparation</u>

Fashion MNIST dataset is already loaded as a NumPy array, there's no need to check file extensions like we **would do** for actual image files. However, we can still verify whether all elements in X_train, X_test, and Y_train are valid image data.

* Clean data.
    * Steps to Validate Images in Fashion MNIST
        * Check the data type → Images should be numeric arrays (uint8 type).
        * Check the shape of each image → All images should be 28×28 pixels.
        * Check for corrupted images (e.g., containing NaN values).

In [ ]:
def check_images(dataset, dataset_name):
    """
    Checks images for:
    * being an array
    * shape (28x28)
    * colour channel values
    * NaN values
    """
    invalid_count = 0  # Counter for invalid images
    valid_count = 0     # Counter for valid images

    for idx, image in enumerate(dataset):
        # Check if the image is a NumPy array
        if not isinstance(image, np.ndarray):
            print(f"{dataset_name} - Index {idx}: Not a valid image array")
            invalid_count += 1
            continue

        # Check shape (should be 28x28)
        if image.shape != (28, 28):
            print(f"{dataset_name} - Index {idx}: Incorrect shape {image.shape}")
            invalid_count += 1
            continue

        # Check if values are within expected range (0-255 for grayscale images)
        if not (image.dtype == np.uint8 and image.min() >= 0 and image.max() <= 255):
            print(f"{dataset_name} - Index {idx}: Invalid pixel values (Min: {image.min()}, Max: {image.max()})")
            invalid_count += 1
            continue

        # Check for NaN values
        if np.isnan(image).any():
            print(f"{dataset_name} - Index {idx}: Contains NaN values")
            invalid_count += 1
            continue

        valid_count += 1

    print(f"\n{dataset_name}: {valid_count} valid images, {invalid_count} invalid images")


In [ ]:
# Run checks on both datasets
print("Checking Images...\n")
check_images(X_train, "Train")
check_images(X_test, "Test")

* Further data splitting: from the **train** set, we split a **validation** set. We set the validation set as 20% of the train set)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
                                    X_train,
                                    y_train,
                                    test_size=0.2,
                                    random_state=0
                                    )

print("* Train set:", X_train.shape, y_train.shape)
print("* Validation set:",  X_val.shape, y_val.shape)
print("* Test set:",   X_test.shape, y_test.shape)

---

### <u>EDA</u>

* Plot label frequency distribution in train, validation and test dataset

In [ ]:
# Define class names. Fasion MNIST has 10 labels
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# Create a DataFrame for label frequency distribution
df_freq = pd.DataFrame(columns=['Set', 'Label', 'Frequency'])


In [ ]:
def count_labels(dataset, dataset_name):
    """
    Helper function to count occurrences of each label and print them
    """
    global df_freq
    unique, counts = np.unique(dataset, return_counts=True)  # Get label frequencies
    for label, frequency in zip(unique, counts):
        df_freq = pd.concat([df_freq, pd.DataFrame([{'Set': dataset_name, 'Label': class_names[label], 'Frequency': frequency}])], ignore_index=True)
        print(f"* {dataset_name} - {class_names[label]}: {frequency} images")  # Print formatted output

In [ ]:
count_labels(y_train, "Train")
count_labels(y_test, "Test")
count_labels(y_val, "Validation")

In [ ]:
# Visualize the label distribution and save image
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))
sns.barplot(data=df_freq, x='Set', y='Frequency', hue='Label')
plt.xticks(rotation=45)
plt.title("Label Frequency Distribution in Train, Validation, and Test Sets")
plt.savefig("/workspaces/m32895-coursework-2025/outputs/label_distribution_in_sets.png", bbox_inches='tight', dpi=150)
plt.show()

At this point we can ask ourselves:
* Is the dataset <u>balanced</u>?
* Do we have enough data?
* Are we going to run **augmentation**?


#### * We need to **reshape** and **rescale** the data to make it digestible by Tensorflow

In [ ]:
# Current data shape:
X_train.shape

* When using Convolutional Neural Networks (CNNs), the Fashion MNIST dataset needs to be reshaped to include a "channel" dimension because CNNs typically expect 4D input

| Dataset Type | Required Shape for CNN |
|--------------|------------------------|
| Grayscale Images (e.g. Fashion MNIST) | (num_images, height, width, 1) |
| RGB Images | (num_images, height, width, 3) |

N.b.: we are taking these steps for scaling the data and reshaping to include the channel dimension since the data was provided in such format and is in a NumPy array format

When you get image datasets in a NumPy format, you will recheck these items, and if required, you will need to process them.
However, when dealing with real images, the preprocessing tasks are done in another way, which we will cover in the walkthrough of project 1

In [ ]:
# Reshape Fashion MNIST data for CNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

# Check the new shape
print(X_train.shape)  # Expected output: (48000, 28, 28, 1)


* Convert to **float** and reshape.

In [ ]:
X_train.max()

In [ ]:
X_train = X_train.astype("float32") / 255.0
X_val = X_val.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

In [ ]:
X_train.max()

* What Happens After Scaling?

| Before Scaling (uint8) | After Scaling (float32) |
|---|---|
| Pixel values: 0 - 255 | Pixel values: 0.0 - 1.0 |
| Integer type (uint8) | Floating-point (float32)|
| Can cause instability in training | Helps stable and faster training |
* The model learns better and converges faster when inputs are scaled to [0,1].



In [ ]:
# **Convert labels to categorical format**
n_labels = 10  # Fashion MNIST has 10 classes
y_train = to_categorical(y_train, num_classes=n_labels)
y_val = to_categorical(y_val, num_classes=n_labels)
y_test = to_categorical(y_test, num_classes=n_labels)

In [ ]:
y_test

---

### <u>Building our model</u>

* Finally! We are about to create a TensorFlow model

* We create a function that
    * creates a sequential model
    * compiles the model
    * returns the model
    
* The function needs the input shape (image size) as well as the number of neurons in the last layer
* The network has two pairs of Convolution + Pooling layers. It is know upfront that for this dataset that one pair would be enough; however, we want to try in this work different combination of layers and their parameters for educational purposes.
* <u>Brief summary</u> (refer to previous tutorials for more details):
    * **Convolution layers** are used to select the dominant pixel value from within images using filters
    * **Pooling layers** reduce the image size by extracting only the dominant pixels
* The first pair has a convolution layer with 16 filters and a kernel size of 3x3.
* We parse the input shape as well as the **relu** as an activation function.
* The **MaxPool** has a pool size of 2x2
* The next pair has the same setup as the previous pair
* Next, there is a Flatten layer
    * The Flatten layer is used to flatten the matrix into a vector, which means a single list of all values. Then that is fed into a dense layer.
* Next, there is a Dense layer with 128 neurons.


* Typically, here, you arrange the dense layers in multiples of 2, and the number of layers depends on the data complexity after the Flatten layer.
* We will check in the `.summary()` or `.plot_model()` that the data shape after the Flatten layer is 400, so it makes sense to reduce the number of neurons from this case, 400 to 128. So naturally, you will only know the output from the Flatten layer is 400 after creating a model and checking the summary/plot_model.
If the output from the Flatten layer were much higher, like 5k, you would consider two or more dense layers to reduce the number of connections progressively.
* The value 128 is a good starting point. If you notice the CNN is not learning, you may add more dense layers and adjust the number of neurons in them
* After, we have a dropout layer with a rate of 25% to reduce the chance of overfitting.
* **The output layer should reflect a multiclass classification.**
* We set a dense layer where the number of neurons equals the number of classes in the target variable. This information is stored in a previously created variable - `n_labels`.
* For multiclass classification, we set the activation function as softmax, and we compile the model with adam as the optimizer and the loss function as categorical_crossentropy.
* We also arranged to monitor the metric accuracy.

In [ ]:
def build_tf_model(input_shape, n_labels):
  model = Sequential()

  model.add(Conv2D(filters=16, kernel_size=(3,3),input_shape=input_shape, activation='relu',))
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu',))
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Flatten())

  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.25))

  model.add(Dense(n_labels, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

In [ ]:
model = build_tf_model(input_shape=X_train.shape[1:], n_labels=n_labels )
model.summary()

---

### <u>Fit the model</u>

* Early stopping allows us to stop the training when a monitored metric has stopped improving
    * This is useful to avoid **overfitting** the model to the data.

* We will monitor the validation accuracy now
    * We set patience as 1, the number of epochs with no improvement, after which training will be stopped
    * There is no fixed rule to set patience
    * If you feel that your model is learning still and you stopped, you may increase the value and train again
    * However, we want the training process to be quick, (for demo purpose, n/a in your course work)
    * We set the mode to min since now we want the model to stop training when the loss didn't improve its performance and improve means decrease
* We will finally fit the model
    * We create the model object and use `.fit()`
    * We parse the Train set
    * The epochs are set to 4. We know in advance that this amount is fine to learn the patterns considering the dataset and the network structure
    * We parse the validation data in a tuple
    * Verbose is set to 1, so we can see in which epochs we are and the training and validation loss
    * Finally, we parse our callback as the early_stop object we created earlier
* For each epoch, note the training and validation loss and accuracy. Is it increasing? Decreasing? Static?
* Ideally, the loss should decrease as long as the epoch increases, showing a practical sign the network is learning. The accuracy should increase over the epochs.
* Note the model will take a bit longer now to train

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1)

In [ ]:
model = build_tf_model(input_shape= X_train.shape[1:], n_labels=n_labels )

model.fit(x=X_train,
          y=y_train,
          epochs=4,
          validation_data=(X_val, y_val),
          verbose=1,
          callbacks=[early_stop]
          )

---

### <u>Model evaluation</u>

* Evaluate the model performance by analysing the training and the validation losses and accuracy that happened during the training process
* In deep learning, we use the model history to assess if the model learned, using the train and validation sets
* We also evaluate separately how the model **generalises** on unseen data (on the test set)
* The model training history information is stored in a .history.history attribute from the model
* Note it shows loss and accuracy for train and validation

In [ ]:
history = pd.DataFrame(model.history.history)
history.head()

#### Plot accuracy and loss

In [ ]:
sns.set_style("whitegrid")
history[['loss','val_loss']].plot(style='.-')
plt.title("Loss")
plt.show()

print("\n")
history[['accuracy','val_accuracy']].plot(style='.-')
plt.title("Accuracy")
plt.show()

* We will evaluate the model performance on the test set, using `.evaluate()` and parsing the test set
* N.b. the value is not much different from the losses and accuracy in the train and validation set
* N.b. the loss is low and accuracy is high. It looks like the model has learned the relationships between the features and the target.

In [ ]:
model.evaluate(X_test,y_test)

* In classification, you would analyse the confusion matrix and classification report using the custom function we have seen over the course.

In [ ]:
def confusion_matrix_and_report(X,y,pipeline,label_map):
  """
  Print confusion matrix and report
  """
  # the prediction comes in a one hot encoded format
  prediction = pipeline.predict(X)
  # so we take the index from the highest probability, which is the "winner" or predicted class
  prediction = np.argmax(prediction, axis=1)

  # we also take the index from the highest probability from the actual values
  y = np.argmax(y, axis=1)

  print('---  Confusion Matrix  ---')
  print(pd.DataFrame(confusion_matrix(y_true=prediction, y_pred=y),
        columns=[ ["Actual " + sub for sub in label_map] ],
        index= [ ["Prediction " + sub for sub in label_map ]]
        ))
  print("\n")

  print('---  Classification Report  ---')
  print(classification_report(y, prediction, target_names=label_map),"\n")


In [ ]:
def clf_performance(X_train,y_train,X_test,y_test,X_val, y_val,pipeline,label_map):
  """
  Print classification performance
  """
  print("#### Train Set #### \n")
  confusion_matrix_and_report(X_train,y_train,pipeline,label_map)

  print("#### Validation Set #### \n")
  confusion_matrix_and_report(X_val,y_val,pipeline,label_map)

  print("#### Test Set ####\n")
  confusion_matrix_and_report(X_test,y_test,pipeline,label_map)

In [ ]:
clf_performance(X_train, y_train,
                X_test,y_test,
                X_val, y_val,
                model,
                label_map= class_names
                )

---

### <u>Prediction</u>

* Let's take a sample from the test set and use it as if it was live data. We will consider 1 sample

In [ ]:
index = 102
my_garment = X_test[index]
class_index = np.argmax(y_test[index])
print(my_garment.shape)
print(y_test[index])
print(f"This is '{class_names[class_index]}'")

sns.set_style('white')
plt.imshow(my_garment.reshape(28,28), cmap='gray')
plt.show()

In [ ]:
my_garment.shape

* However when interacting with the **model**, we need the data in 4 dimensions, where the first dimension is the number of images the data has, the next 2 are the image size and the last is the color channels
* In our case, we need to add the first dimension, and the value will be 1, so the final shape is (1 ,28 ,28 ,1 )
* We use the command  `np.expand_dims()` for this task

In [ ]:
live_data = np.expand_dims(my_garment, axis=0)
print(live_data.shape)

* We use `.predict()` and parse the data
* N.b. the result is a *probabilistic* result for each class.

In [ ]:
prediction_proba = model.predict(live_data)
prediction_proba

* So we take the index from the highest probability, which is the "winner" or predicted class

In [ ]:
prediction_class = np.argmax(prediction_proba, axis=1)
prediction_class

* Plot the probabilistic result so that you can check the predictions in a more visual fashion
* Read the pseudo-code
* In the end you are getting prediction_proba to define the associate probability for each class
* Then you plot it in a bar plot using Plotly

In [ ]:
# create an empty dataframe, that will show the probability per class
# we set the probabilities as the prediction_proba
prob_per_class= pd.DataFrame(data=prediction_proba[0],
                             columns=['Probability']
                             )

# we round the values to 3 decimal points, for better visualization
prob_per_class = prob_per_class.round(3)

# we add a column to prob_per_class that shows the meaning of each class
# in this case, the species name that is mapped in the target_classes
prob_per_class['Results'] = class_names

prob_per_class

#### Plot prediction probability for each garment in the dataset

In [ ]:
fig = px.bar(
        prob_per_class,
        x = 'Results',
        y = 'Probability',
        range_y=[0,1],
        width=600, height=400,template='seaborn')
fig.update_xaxes(type='category')
fig.show()